## Dataset: Mirage22

### Augmentations: 
- Packet loss
- Flows with less than 10 packets removed

In [ ]:
# -- IMPORTS --

import os
os.chdir('/workplace/flowmind/')

import csv
import sys

# torch imports
import torch
import torch.nn as nn
from torcheval.metrics import MulticlassAccuracy

from flowmind.contrastive import NTXentLoss

# livelossplot imports
from livelossplot import PlotLosses
from livelossplot.outputs import MatplotlibPlot

# others
from functools import partial

# common imports
os.chdir('/workplace/xcocek00/common/')
from augmentations import augment_rtt
os.chdir('/workplace/xcocek00/common/')
from dataloader import create_flowpic_dataloader
os.chdir('/workplace/xcocek00/common/')
from nn import CNN, MLP

# sets csv limit
csv.field_size_limit(sys.maxsize)

# sets the device to use gpu if available, if not, use cpu
device = torch.device(f"cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
# -- custom remap_label function --

def remap_label(t: tuple[torch.Tensor, torch.Tensor, str], labels_d: dict[str, int]) -> tuple[torch.Tensor, torch.Tensor, int]:
    return t[0], t[1], labels_d[t[2]]

labels_mirage22 = [
    "com.Slack",
    "com.cisco.webex.meetings",
    "com.discord",
    "com.facebook.orca",
    "com.google.android.apps.meetings",
    "com.gotomeeting",
    "com.microsoft.teams",
    "com.skype.raider",
    "us.zoom.videomeetings",
]


labels_mirage22_d = {label: i for i, label in enumerate(labels_mirage22)}

remap_label_mirage22 = partial(remap_label, labels_d=labels_mirage22_d)



In [ ]:
# -- DATA LOADERS for Mirage22 --
# dataloaders yield tuple (flowpic1, flowpic2, label)
# using only 15 seconds of the flow

# 0 - debug prints are turned off
# 1 - debug prints are turned on
DEBUG = 0

# dataloader used for training
dl_train = create_flowpic_dataloader(
    dir_path="/workplace/datasets/mirage22/processed/splits-10/train.csv",
    batch_size=32,
    meta_key="BF_label",
    time_bins = [i * (15 / 32) for i in range(33)],
    length_bins = [i * (1500 / 32) for i in range(33)],
    flow_transform_1=augment_rtt,
    flow_transform_2=augment_rtt,
    bidirectional = False,
    min_packets=10,
    #min_length=25,
    dp_transform=lambda dp: dp.map(remap_label_mirage22).in_memory_cache(),
)

# dataloader used for validation
dl_val = create_flowpic_dataloader(
    dir_path="/workplace/datasets/mirage22/processed/splits-10/val.csv",
    batch_size=32,
    meta_key="BF_label",
    time_bins = [i * (15 / 32) for i in range(33)],
    length_bins = [i * (1500 / 32) for i in range(33)],
    flow_transform_2=augment_rtt,
    flow_transform_1=augment_rtt,
    bidirectional = False,    
    min_packets=10,
    #min_length=25,
    dp_transform=lambda dp: dp.map(remap_label_mirage22).in_memory_cache(),
)

# dataloader used for classification head
dl_class = create_flowpic_dataloader(
    dir_path="/workplace/datasets/mirage22/processed/splits-10/train.csv",
    batch_size=32,
    meta_key="BF_label",
    time_bins = [i * (15 / 32) for i in range(33)],
    length_bins = [i * (1500 / 32) for i in range(33)],
    bidirectional = False,   
    min_packets=10,
    #min_length=25,
    dp_transform=lambda dp: dp.map(remap_label_mirage22).in_memory_cache(),
)

# dataloader used for unseen data to test model
dl_test = create_flowpic_dataloader(
    dir_path="/workplace/datasets/mirage22/processed/splits-10/test.csv",
    batch_size=32,
    meta_key="BF_label",
    time_bins = [i * (15 / 32) for i in range(33)],
    length_bins = [i * (1500 / 32) for i in range(33)],
    bidirectional = False,   
    min_packets=10,
    #min_length=25,
    dp_transform=lambda dp: dp.map(remap_label_mirage22).in_memory_cache(),
)

if DEBUG:    
    #print(type(dl_train))
    #dim_val(dl_train)
    #print_batch(dl_train, "training")
    #print_batch(dl_class, "classification")
    #debug_batch(dl_train)
    print("debug")

In [ ]:
# -- training loops--

def train(model, dataloader, optimizer, contrastive_loss_fn, log):
    '''
    Training loop

    Args:
        model: CNN model
        dataloader: pytorch dataloader created by create_flowpic_dataloader function
        optimizer: chosen optimizer
        contrastive_loss_fn: contrastive loss function (NT-Xent)
        log: for log collection

    Returns:
        Avarage loss 
    '''

    model.train()
    total_loss = 0.0
    batches = 0

    # label not needed in -> _
    for flowpic1, flowpic2, _ in dataloader:
        flowpic1, flowpic2 = flowpic1.to(device), flowpic2.to(device)

        optimizer.zero_grad()

        z1 = model(flowpic1)
        z2 = model(flowpic2)
        
        # contrastive loss
        loss = contrastive_loss_fn(z1, z2)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        batches += 1
        
    total_loss /= batches
    log["loss"] = total_loss

    return total_loss

def val(model, dataloader, contrastive_loss_fn, log):
    '''
    Validation loop

    Args:
        model: CNN model
        dataloader: pytorch dataloader created by create_flowpic_dataloader function
        contrastive_loss_fn: contrastive loss function (NT-Xent)
        log: for log collection

    Returns:
        Avarage loss 
    '''
    with torch.no_grad():
        model.eval()
        total_loss = 0.0
        batches = 0

        # label not needed in -> _
        for flowpic1, flowpic2, _ in dataloader:
            flowpic1, flowpic2 = flowpic1.to(device), flowpic2.to(device)

            z1 = model(flowpic1)
            z2 = model(flowpic2)
            
            # contrastive loss
            loss = contrastive_loss_fn(z1, z2)

            total_loss += loss.item()
            batches += 1

        total_loss /= batches
        log["val_loss"] = total_loss

    return total_loss

def classification(CNN_model, MLP_model, dataloader, log, loss_fn, accuracy, optimizer):
    '''
    Classification loop

    Args:
        CNN_model: CNN model 
        MLP_model: MLP model
        dataloader: pytorch dataloader created by create_flowpic_dataloader function
        log: for log collection
        loss_fn: chosen loss function
        accuracy: chosen model accuracy
        optimizer: chosen optimizer

    Returns:
        classification loss 
    '''

    # freeze the encoder
    for param in CNN_model.encoder.parameters():
        param.requires_grad = False

    # remove projection head
    CNN_model.projection = nn.Sequential()
    
    # set models
    CNN_model.eval()
    MLP_model.train()

    classification_loss = 0.0
    batches = 0

    for flowpic, _, label in dataloader:
        optimizer.zero_grad()

        flowpic, label = flowpic.to(device), label.to(device)

        with torch.no_grad():
            embeddings = CNN_model.encoder(flowpic)
        
        y_pred = MLP_model(embeddings)

        loss = loss_fn(y_pred, label)
        loss.backward()
        optimizer.step()

        classification_loss += loss.item()
        batches += 1
        
        accuracy.update(y_pred, label)
    
    classification_loss /= batches

    log["classification_loss"] = classification_loss
    log["accuracy"] = accuracy.compute()
        
    accuracy.reset()

    return classification_loss

def testing(CNN_model, MLP_model, dataloader, log, loss_fn, accuracy):
    '''
    Testing loop

    Args:
        CNN_model: CNN model 
        MLP_model: MLP model
        dataloader: pytorch dataloader created by create_flowpic_dataloader function
        log: for log collection
        loss_fn: chosen loss function
        accuracy: chosen model accuracy
        optimizer: chosen optimizer

    Returns:
        training loss 
    '''

    # freeze the encoder
    for param in CNN_model.encoder.parameters():
        param.requires_grad = False
    
    # set models
    CNN_model.eval()
    MLP_model.eval()

    test_loss = 0.0
    batches = 0

    with torch.no_grad():
        for flowpic_t, _, label in dataloader:
            flowpic_t, label = flowpic_t.to(device), label.to(device)

            embeddings = CNN_model.encoder(flowpic_t)
            y_pred = MLP_model(embeddings)

            loss = loss_fn(y_pred, label)
            test_loss += loss.item()
            batches += 1

            accuracy.update(y_pred, label)
    
        test_loss /= batches
    
        log["model_loss"] = test_loss
        log["model_accuracy"] = accuracy.compute()
            
        accuracy.reset()

    return test_loss

In [ ]:
# -- Mirage22 dataset --

mirage22_classes = 9

# CNN definition
cnn_model = CNN()
cnn_model = cnn_model.to(device)

# optimizer
cnn_optimizer = torch.optim.Adam(cnn_model.parameters(), lr=0.001)

# NT-Xent loss function
contrastive_loss_fn = NTXentLoss(temperature=0.07)

cnn_liveloss = PlotLosses(outputs=[MatplotlibPlot(cell_size=(6, 2))])

# Linear classifier definition
mlp_model = MLP(mirage22_classes)
mlp_model = mlp_model.to(device)

# optimizer
mlp_optimizer = torch.optim.Adam(mlp_model.parameters(), lr=0.01)

# loss function
mlp_loss_fn = nn.CrossEntropyLoss()

mlp_accuracy = MulticlassAccuracy()

mlp_liveloss = PlotLosses(outputs=[MatplotlibPlot(cell_size=(6, 2))])

In [ ]:
# -- traning CNN --

for epoch in range(15):
    log = {}
    train_loss = train(cnn_model, dl_train, cnn_optimizer, contrastive_loss_fn, log)
    val_loss = val(cnn_model, dl_val, contrastive_loss_fn, log)

    cnn_liveloss.update(log)
    cnn_liveloss.send()

In [ ]:
# -- training linear classifier --

# set params for early stopping
best_loss = float('inf')
no_improvement = 0
min_improvement = 0.001
patience = 5

for epoch in range(20):
    log = {}
    classification_loss = classification(cnn_model, mlp_model, dl_class, log, mlp_loss_fn, mlp_accuracy, mlp_optimizer)   
    test = testing(cnn_model, mlp_model, dl_test, log, mlp_loss_fn, mlp_accuracy)   

    mlp_liveloss.update(log)
    mlp_liveloss.send()

    # early stopping
    if best_loss - classification_loss > min_improvement:
        best_loss = classification_loss
        no_improvement = 0
    else:
        no_improvement += 1

    if no_improvement >= patience:
        print("Early stopping - epoch ", epoch)
        break
    

In [ ]:
print("Model accuracy: ", mlp_accuracy)